In [1]:
# Base Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import glob
from collections import defaultdict


In [2]:
csv_files = glob.glob('D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12/*.csv')
selected_files = csv_files[:]
print(selected_files)

['D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\\DrDoS_DNS.csv', 'D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\\DrDoS_LDAP.csv', 'D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\\DrDoS_MSSQL.csv', 'D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\\DrDoS_NetBIOS.csv', 'D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\\DrDoS_NTP.csv', 'D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\\DrDoS_SNMP.csv', 'D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\\DrDoS_SSDP.csv', 'D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\\DrDoS_UDP.csv', 'D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\\Syn.csv', 'D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\\TFTP.csv', 'D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\\UDPLag.csv']


In [ ]:
label_limit = 50000  # jumlah maksimum per label
label_counts = defaultdict(int)
df_list = []

for file in selected_files:
    print(f"Processing file: {file}")
    
    # Baca seluruh isi file tanpa chunk
    df = pd.read_csv(file, low_memory=False)
    
    # Acak data agar setiap baris punya peluang sama
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Filter baris yang belum mencapai batas jumlah per label
    filtered_df = df[df[' Label'].apply(lambda label: label_counts[label] < label_limit)]

    # Update counter dan simpan data
    for label in filtered_df[' Label'].unique():
        count_in_df = (filtered_df[' Label'] == label).sum()
        remaining = label_limit - label_counts[label]
        take = min(count_in_df, remaining)
        if take > 0:
            selected_rows = filtered_df[filtered_df[' Label'] == label].head(take)
            df_list.append(selected_rows)
            label_counts[label] += take

# Gabungkan semua data
combined_df = pd.concat(df_list, ignore_index=True)
print(combined_df.shape)

Processing file: D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\DrDoS_DNS.csv
Processing file: D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\DrDoS_LDAP.csv
Processing file: D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\DrDoS_MSSQL.csv
Processing file: D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\DrDoS_NetBIOS.csv
Processing file: D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\DrDoS_NTP.csv
Processing file: D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\DrDoS_SNMP.csv
Processing file: D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\DrDoS_SSDP.csv
Processing file: D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\DrDoS_UDP.csv
Processing file: D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\Syn.csv
Processing file: D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\TFTP.csv
Processing file: D:/J/EnsembleAsli/CICDDoS-Real/CSV-01-12/01-12\UDPLag.csv
(600439, 88)


In [4]:
combined_df[' Label'].value_counts()

 Label
DrDoS_DNS        50000
BENIGN           50000
DrDoS_LDAP       50000
DrDoS_MSSQL      50000
DrDoS_NetBIOS    50000
DrDoS_NTP        50000
DrDoS_SNMP       50000
DrDoS_SSDP       50000
DrDoS_UDP        50000
Syn              50000
TFTP             50000
UDP-lag          50000
WebDDoS            439
Name: count, dtype: int64

In [ ]:
# Buang kelas yang tidak termasuk (WebDDoS)
combined_df = combined_df[combined_df[' Label'] != 'WebDDoS']

In [5]:
# Simpan ke CSV
combined_df.to_csv('RandomSampling_cicddos2019.csv', index=False)
print("File saved as RandomSampling_cicddos2019.csv")

File saved as RandomSampling_cicddos2019.csv
